In [1]:
import requests

In [2]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [40]:
data = requests.get(standings_url)


In [4]:
from bs4 import BeautifulSoup

In [34]:
soup = BeautifulSoup(data.text)


In [41]:
standings_table = soup.select('table.stats_table')[0]


IndexError: list index out of range

In [8]:
links = standings_table.find_all('a')

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [ ]:
links[0].get("href")

In [ ]:
links = [l.get("href") for l in links]


In [ ]:
team_urls = [f"http://fbref.com{l}" for l in links]

In [ ]:
team_url = team_urls[0]

In [ ]:
data = requests.get(team_url)

In [ ]:
import pandas as pd

matches = pd.read_html(data.text, match="Scores & Fixtures")

In [ ]:
matches[0].head()

In [ ]:
soup = BeautifulSoup(data.text)

In [ ]:
links = soup.find_all('a')

In [ ]:
links = [l.get("href") for l in links]

In [ ]:
links = [l for l in links if l and "all_comps/shooting/" in l]

In [ ]:
data = requests.get(f"https://fbref.com{links[0]}")

In [ ]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [ ]:
shooting.columns = shooting.columns.droplevel()

In [ ]:
shooting.head()

In [ ]:
team_data = matches[0].merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK","PKatt"]], on="Date")

In [ ]:
team_data.head()


In [ ]:
matches[0].shape


In [ ]:
shooting.shape

In [45]:
years = list(range(2024, 2020, -1))

In [46]:
years

[2024, 2023, 2022, 2021]

In [47]:
all_matches = []

In [48]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [49]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')
    
    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url= f"https://fbref.com/{previous_season}"
    for team_url in team_urls:
        team_name = team_url.split('/')[-1].replace('-Stats', "").replace("-", "")

        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()

        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK","PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)
    
    

[]


AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df.columns = [c.lower for c in match_df.columns]

In [ ]:
match_df.to_csv("../data-scrapped/matches.csv")